# Pre calculate Lambda solutions

Given a $H, E[\beta]$ input, we find the three-dimensional lambda solution.

Find k by k solutions, then find a smart, fast way to do the lookup
The lookup should find the row and column index where the solution lies
and also the four quadrants.

These solution take src.betas_transition as given, so can't be used for
different values of these

In [19]:
import numpy as np
from numba import njit
from scipy.stats import entropy
from scipy import optimize

#Constants
betas_transition = np.array([-4., -2., -1.1])


def reparam_lambdas(x):
    """
    uses softmax to get values between 0 and 1
    and make them sum to 1
    """
    #return np.exp(x) / np.sum(np.exp(x))
    #Numerically more stable version
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


#@njit()
def h_and_exp_betas_eqns(orig_lambdas, βs, Eβ, H, w=np.array([[1., 0.], [0., 1./4.]])):
    """
    orig_lambdas: original lambda tries (not summing to zero, not within [0, 1])
    Eβ, H: the objectives
    βs: fixed constant of the model
    """
    lambdas = reparam_lambdas(orig_lambdas)
    g = np.array([entropy(lambdas) - H, np.dot(βs, lambdas) - Eβ])
    return g.T @ w @ g


def H_and_eb_to_lambda0(H, Eβ,
                        starting_values=np.array([0.1, 1.5, 1.])):
    """
    Generates a lambda0 vector from the values of
    the entropy and expected value of betas (H, EB)
    """

    def fun_(lambda_try):
        return h_and_exp_betas_eqns(lambda_try, betas_transition, Eβ, H)

    sol = optimize.minimize(fun_, x0=starting_values, method='Powell')
    lambdas_sol = reparam_lambdas(sol.x)
    if not sol.success:
        # Use Nelder-Mead from different starting_value
        sol = optimize.minimize(fun_, x0=np.array([1.6, 0.1, 0.25]),
                                method='Nelder-Mead')
        lambdas_sol = reparam_lambdas(sol.x)
        if not sol.success:
            print(f"Theta to lambda0 didn't converge", sol.x, lambdas_sol)

    return lambdas_sol

In [62]:
h_bounds = [0, 1]
eb_bounds = [betas_transition[0], betas_transition[-1]]

h_digit_precision = 0.01
eb_digit_preciision = 0.01
h_candidates = np.arange(h_bounds[0], h_bounds[1], h_digit_precision)
eb_candidates = np.arange(eb_bounds[0], eb_bounds[1], eb_digit_preciision)
grid_size = (len(h_candidates), len(eb_candidates))
grid_total_elements = grid_size[0]*grid_size[1]
lambda_values = np.empty((grid_size[0], grid_size[1], 3))
print(grid_size)

(100, 290)


In [61]:
import time
start = time.time()

for i_h, h in enumerate(h_candidates):
    for i_eb, eb in enumerate(eb_candidates):
        lambda_values[i_h, i_eb, :] = H_and_eb_to_lambda0(h, eb)
        
secs_it_took = time.time() - start

In [29]:
secs_por_element =  secs_it_took / grid_total_elements 
print(f"tomó {secs_por_element} minutos por valor")

tomó 0.009705712985992432 minutos por valor


In [68]:
h_n_digits_precision = 3
eb_n_digits_precision = 3
h_dict = {}
for i in range(len(h_candidates)):
    h_dict[np.round(h_candidates[i], h_n_digits_precision)] = i

e_dict = {}
for i in range(len(eb_candidates)):
    e_dict[np.round(eb_candidates[i], eb_n_digits_precision)] = i

In [71]:
h_dict.keys()

dict_keys([0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [82]:
candidate = 0.33*0.3 + 0.34*0.7
h_dict[np.round(candidate, h_n_digits_precision-1)]

34

In [88]:
list(e_dict.keys())[0:5]

[-4.0, -3.99, -3.98, -3.97, -3.96]

In [90]:
e_dict[np.round(-3.9827171, 2)]


2

In [101]:
#Input
eb, H = -3.977, 0.2134

#Find row, col
row = h_dict[np.round(H, h_n_digits_precision-1)]
col = e_dict[np.round(eb, eb_n_digits_precision-1)]

# Distances to row-1, row, row+1 and col-1, col, col+1
dist_row = np.array([np.abs(H - h_candidates[row - 1]), np.abs(H - h_candidates[row]), 
           np.abs(H - h_candidates[row + 1])])
if np.argmax(dist_row) == 2:
    relevant_rows = [row-1, row]
    dist_row = dist_row[0:2]
elif np.argmax(dist_row) == 0:
    relevant_rows = [row, row+1]
    dist_row = dist_row[1::]

dist_col = np.array([np.abs(eb- eb_candidates[col - 1]), np.abs(eb - eb_candidates[col]), 
           np.abs(eb - eb_candidates[col + 1])])
if np.argmax(dist_col) == 2:
    relevant_cols = [col-1, col]
    dist_col = dist_col[0:2]
elif np.argmax(dist_col) == 0:
    relevant_cols = [col, col+1]
    dist_col = dist_col[1::]

    


{1, 2}

In [105]:
def inv_dis(da, db):
    return (1/da) / ((1/da) + (1/db))

inv_dis(0.02, 0.1)

0.8333333333333334

0.017171717

In [45]:
np.round(h_candidates[1], 3)

0.01

In [46]:
h_candidates

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747

In [32]:
(secs_por_element * 1000000) / 60

161.76188309987387

In [10]:
def avers(x):
    return np.exp(x) / np.sum(np.exp(x))

avers([0.999, 0.001, 0.001])

array([0.5756284, 0.2121858, 0.2121858])

In [5]:
np.array([0.26030255, 0.38832577, 0.35137169]).sum()

1.00000001

In [6]:
np.array([0.1, 0.5, 0.4])

array([0.1, 0.5, 0.4])

In [11]:
logit(np.array([0.5756284, 0.2121858, 0.2121858]))

array([ 0.30485283, -1.31179997, -1.31179997])

In [12]:
avers(np.array([0.1, 1.5, 1.]))

array([0.13307069, 0.53962824, 0.32730107])